In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# ✅ ตั้งค่าพารามิเตอร์การเทรด
capital = 10000  # เงินทุนเริ่มต้น
leverage = 100  # เลเวอเรจ
lot_size = 0.1  # ขนาด Lot
pip_value = 0.1  # ราคาของ 1 Pip ใน XAUUSD
target_profit = capital * 0.1  # เป้าหมายกำไร 10% ของทุน ($1,000)

# ✅ โหลดข้อมูลราคาทองคำและ Grid Price
gold_df = pd.read_csv("mt5_data_XAUUSD_TF1H_jan_TH.csv")
grid_df = pd.read_csv("pricegrid_2mount.ipynb_reordered.csv")

# ✅ แปลงคอลัมน์ Date เป็น datetime
gold_df['Date'] = pd.to_datetime(gold_df['Date'], errors='coerce')
grid_df['Date'] = pd.to_datetime(grid_df['Date'], errors='coerce')

# ✅ ลบค่าที่เป็น NaN
gold_df = gold_df.dropna(subset=['Date', 'Close'])
grid_df = grid_df.dropna(subset=['GridPrice'])

# ✅ สร้างลิสต์เก็บ Buy Orders
buy_orders = []
total_profit = 0  # กำไรรวมทั้งหมด

print("\n--- เริ่ม Loop ตรวจสอบ Signal Buy ---\n")

# ✅ Loop ตรวจสอบ Signal Buy
for i in range(1, len(gold_df)):
    close_prev = gold_df.iloc[i - 1]["Close"]
    close_current = gold_df.iloc[i]["Close"]
    date_current = gold_df.iloc[i]["Date"]

    # ✅ ดึง Grid Data ที่เป็นอดีตเท่านั้น (Grid Date ต้อง <= Date ปัจจุบัน)
    valid_grids = grid_df[grid_df["Date"] <= date_current]

    if valid_grids.empty:
        print(f"⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ {date_current}")
        continue

    # ✅ ดึง Grid Prices ทั้งหมดในวันนั้น
    grid_prices = valid_grids["GridPrice"].values
    grid_dates = valid_grids["Date"].values  # ดึง Grid Date ทุกอันด้วย

    # 🔎 Debug log
    print(f"\nตรวจสอบข้อมูล: {date_current}")
    print(f"   ➤ Close Prev: {close_prev}")
    print(f"   ➤ Close Current: {close_current}")
    print(f"   ➤ Grid Prices: {grid_prices}")

    # ✅ ตรวจสอบว่า Close แตะ GridPrice ในวันนั้นๆ
    for grid_price, grid_date in zip(grid_prices, grid_dates):
        if min(close_prev, close_current) <= grid_price <= max(close_prev, close_current):
            print(f"   ✅ ตรงเงื่อนไข! เกิด Signal Buy ที่ Grid Price: {grid_price}, Grid Date: {grid_date}")

            # ✅ บันทึกออเดอร์ Buy
            buy_orders.append({
                "Buy Date": date_current,
                "Buy Price": close_current,
                "Grid Price": grid_price,
                "Grid Date": grid_date,
                "Lot Size": lot_size,
                "Status": "Holding"
            })

            # ✅ ลบ Grid Price ที่ใช้แล้วออกจาก DataFrame
            grid_df = grid_df[~((grid_df["GridPrice"] == grid_price) & (grid_df["Date"] == grid_date))]

            break  # ถ้าพบการซื้อแล้ว หยุด loop Grid Price ของวันนั้น

    # ✅ คำนวณกำไรรวม
    total_profit = 0
    for order in buy_orders:
        buy_price = order["Buy Price"]
        profit_per_order = (close_current - buy_price) * lot_size * (1 / pip_value)  # คำนวณกำไร
        total_profit += profit_per_order  # รวมกำไรทุกไม้

    print(f"💰 กำไรรวมปัจจุบัน: ${total_profit:.2f}")

    # ✅ เช็คว่าถึงเป้าหมายกำไรหรือยัง
    if total_profit >= target_profit:
        print("\n🎯 กำไรรวมถึงเป้าหมาย $1,000 ➜ ปรับเงินทุนใหม่\n")
        capital += total_profit  # เพิ่มกำไรเข้าไปในเงินทุน
        target_profit = capital * 0.1  # คำนวณเป้าหมายกำไรใหม่ 10% ของเงินทุนใหม่
        print(f"💸 เงินทุนใหม่: ${capital:.2f} -> เป้าหมายกำไรใหม่: ${target_profit:.2f}\n")

        # รีเซ็ตกำไรรวม
        total_profit = 0

# ✅ แปลงเป็น DataFrame และเรียงลำดับตาม Buy Date
if buy_orders:  # ตรวจสอบว่ามีข้อมูลหรือไม่
    buy_orders_df = pd.DataFrame(buy_orders)
    buy_orders_df = buy_orders_df.sort_values(by="Buy Date").reset_index(drop=True)
else:
    buy_orders_df = pd.DataFrame(columns=["Buy Date", "Buy Price", "Grid Price", "Grid Date", "Lot Size", "Status"])

# ✅ แสดงผลลัพธ์ทั้งหมด
print("\n--- Buy Orders ที่ถืออยู่ ---\n")
print(buy_orders_df)

# ✅ Plot กราฟ
plt.figure(figsize=(14, 6))

# Plot เส้นราคาทองคำ
plt.plot(gold_df["Date"], gold_df["Close"], label="Gold Price", color="blue", linewidth=1)

# Plot Grid Price เป็นเส้นแนวนอน
for price in grid_df["GridPrice"].unique():
    plt.axhline(y=price, color="red", linestyle="--", alpha=0.3, label="Grid Price" if price == grid_df["GridPrice"].unique()[0] else "")

# Plot จุดซื้อ (Buy Orders)
if not buy_orders_df.empty:
    plt.scatter(buy_orders_df["Buy Date"], buy_orders_df["Buy Price"], color="green", label="Buy Orders", marker="o", s=80)

# ✅ ตั้งค่ากราฟ
plt.title("Gold Price with Grid Trading Signals")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.xticks(rotation=45)

# ✅ แสดงกราฟ
plt.show()


C:\Users\ไพรัตน์\AppData\Local\Temp\ipykernel_65712\2975533817.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



--- เริ่ม Loop ตรวจสอบ Signal Buy ---

⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2024-12-31 23:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-01 00:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-01 01:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-01 02:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-01 03:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-01 04:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-02 06:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-02 07:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-02 08:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-02 09:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-02 10:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-02 11:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-02 12:00:00
⏩ ข้ามไปก่อนเพราะยังไม่มี Grid Data ที่เหมาะสม ณ 2025-01-02 13:

In [2]:
buy_orders_df

Buy Date  Buy Price  Grid Price           Grid Date  Lot Size  \
0  2025-01-02 17:00:00    2642.66     2639.50 2025-01-02 14:00:00       0.1   
1  2025-01-06 13:00:00    2631.53     2632.41 2025-01-03 14:00:00       0.1   
2  2025-01-10 20:00:00    2682.72     2682.41 2025-01-03 14:00:00       0.1   
3  2025-01-10 21:00:00    2688.04     2686.93 2025-01-06 14:00:00       0.1   
4  2025-01-10 22:00:00    2692.73     2689.50 2025-01-02 14:00:00       0.1   
5  2025-01-11 01:00:00    2687.26     2689.46 2025-01-07 14:00:00       0.1   
6  2025-01-13 08:00:00    2685.63     2685.73 2025-01-08 14:00:00       0.1   
7  2025-01-13 17:00:00    2679.50     2684.13 2025-01-09 14:00:00       0.1   
8  2025-01-13 19:00:00    2670.95     2675.90 2025-01-10 14:00:00       0.1   
9  2025-01-15 03:00:00    2676.43     2675.01 2025-01-13 14:00:00       0.1   
10 2025-01-15 16:00:00    2685.89     2684.16 2025-01-14 14:00:00       0.1   
11 2025-01-15 17:00:00    2688.32     2686.79 2025-01-15 14:00:00       0.1   
12 2025-01-17 00:00:00    2721.72     2720.82 2025-01-16 14:00:00       0.1   
13 2025-01-20 14:00:00    2703.47     2708.60 2025-01-20 14:00:00       0.1   
14 2025-01-21 09:00:00    2724.76     2720.77 2025-01-17 14:00:00       0.1   
15 2025-01-21 11:00:00    2727.36     2725.90 2025-01-10 14:00:00       0.1   
16 2025-01-21 13:00:00    2724.42     2725.01 2025-01-13 14:00:00       0.1   
17 2025-01-21 21:00:00    2738.18     2732.41 2025-01-03 14:00:00       0.1   
18 2025-01-21 23:00:00    2743.18     2739.50 2025-01-02 14:00:00       0.1   
19 2025-01-22 17:00:00    2759.87     2758.60 2025-01-20 14:00:00       0.1   
20 2025-01-24 09:00:00    2774.31     2770.82 2025-01-16 14:00:00       0.1   
21 2025-01-24 12:00:00    2775.28     2775.01 2025-01-13 14:00:00       0.1   
22 2025-01-24 18:00:00    2776.66     2775.90 2025-01-10 14:00:00       0.1   
23 2025-01-24 20:00:00    2782.39     2780.81 2025-01-21 14:00:00       0.1   
24 2025-01-27 06:00:00    2767.70     2770.77 2025-01-17 14:00:00       0.1   
25 2025-01-27 22:00:00    2742.10     2742.44 2025-01-24 14:00:00       0.1   
26 2025-01-27 23:00:00    2734.47     2736.93 2025-01-06 14:00:00       0.1   
27 2025-01-28 00:00:00    2733.70     2734.13 2025-01-09 14:00:00       0.1   
28 2025-01-28 01:00:00    2737.02     2735.73 2025-01-08 14:00:00       0.1   
29 2025-01-28 03:00:00    2741.47     2739.46 2025-01-07 14:00:00       0.1   
30 2025-01-28 04:00:00    2740.67     2741.38 2025-01-23 14:00:00       0.1   
31 2025-01-28 19:00:00    2745.53     2745.29 2025-01-28 14:00:00       0.1   
32 2025-01-28 20:00:00    2748.79     2745.78 2025-01-27 14:00:00       0.1   
33 2025-01-30 20:00:00    2785.77     2782.41 2025-01-03 14:00:00       0.1   
34 2025-01-30 21:00:00    2787.79     2786.93 2025-01-06 14:00:00       0.1   
35 2025-01-30 22:00:00    2795.42     2789.50 2025-01-02 14:00:00       0.1   
36 2025-01-30 23:00:00    2790.53     2791.38 2025-01-23 14:00:00       0.1   
37 2025-01-31 00:00:00    2795.61     2792.44 2025-01-24 14:00:00       0.1   
38 2025-01-31 01:00:00    2795.13     2795.29 2025-01-28 14:00:00       0.1   
39 2025-01-31 06:00:00    2796.19     2795.78 2025-01-27 14:00:00       0.1   
40 2025-02-03 08:00:00    2782.20     2789.46 2025-01-07 14:00:00       0.1   
41 2025-02-03 13:00:00    2786.68     2785.73 2025-01-08 14:00:00       0.1   
42 2025-02-03 14:00:00    2789.25     2786.79 2025-01-15 14:00:00       0.1   
43 2025-02-03 15:00:00    2796.65     2791.60 2025-01-29 14:00:00       0.1   

     Status  
0   Holding  
1   Holding  
2   Holding  
3   Holding  
4   Holding  
5   Holding  
6   Holding  
7   Holding  
8   Holding  
9   Holding  
10  Holding  
11  Holding  
12  Holding  
13  Holding  
14  Holding  
15  Holding  
16  Holding  
17  Holding  
18  Holding  
19  Holding  
20  Holding  
21  Holding  
22  Holding  
23  Holding  
24  Holding  
25  Holding  
26  Holding  
27  Holding  
28  Holding  
29  Holding  
30  Holding  
31  Holdin